# SEIRX model runs for no intervention measures

In [1]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

In [2]:
# paths for I/O
contact_network_src = '../data/contact_networks'

## Simulation function

In [3]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''    

    # extract the simulation parameters from the parameter list
    index_case, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params,
                contact_network_src, ensmbl_dst, index_case=index_case)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case)

    return row

## Screening parameters

In [4]:
# specifies, whether the index case will be introduced via an employee or a 
# resident
index_cases = ['employee', 'resident']

# ensemble_size
N_runs = 5000

screening_params = [(i, N_runs) for i in index_cases ]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 2 parameter combinations


## B.1.1.7

### Simulation parameters and measures in place

In [5]:
with open('params/no_intervention_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/no_intervention_measures.json') as json_file:
    measures = json.load(json_file)


In [6]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.109549,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [7]:
measures

{'testing': False,
 'preventive_screening_test_type': None,
 'diagnostic_test_type': None,
 'K1_contact_types': ['close'],
 'quarantine_duration': 0,
 'employee_mask': False,
 'resident_mask': False,
 'employee_screening_interval': None,
 'resident_screening_interval': None,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [8]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/no_measures/UK_variant'

number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_no_measures_UK_variant_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [15:07<00:00, 453.80s/it]

CPU times: user 134 ms, sys: 91.2 ms, total: 225 ms
Wall time: 15min 7s


,index_case,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,N_agents_median,N_agents_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,employee,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,20.451349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,resident,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,17.719676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Wild type

In [9]:
ensmbl_dst = '../data/simulation_results/ensembles/no_measures/wild_type'

### Simulation parameters and measures in place

In [10]:
with open('params/no_intervention_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/no_intervention_measures.json') as json_file:
    measures = json.load(json_file)


In [11]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.073033,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [12]:
measures

{'testing': False,
 'preventive_screening_test_type': None,
 'diagnostic_test_type': None,
 'K1_contact_types': ['close'],
 'quarantine_duration': 0,
 'employee_mask': False,
 'resident_mask': False,
 'employee_screening_interval': None,
 'resident_screening_interval': None,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [13]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_no_measures_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [12:25<00:00, 372.78s/it]

CPU times: user 95.2 ms, sys: 114 ms, total: 209 ms
Wall time: 12min 25s


,index_case,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,N_agents_median,N_agents_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,employee,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,19.907547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,resident,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,0.0,...,19.217050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
